In [1]:
import sys
import config
from config import *
from bs4 import BeautifulSoup
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
import numpy as np
import pickle

In [2]:
data = pd.read_csv('data_clusters.csv')

In [3]:
from config import *
client_credentials_manager = SpotifyClientCredentials(client_id=client_ID, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_ID,
                                                           client_secret=client_secret))

In [13]:
def recommend_songs():
    # Get input from the user
    artist = input("Type in the artist: ")
    title = input("Type in the song name: ")

    # Initialize Spotify API client
    client_credentials_manager = SpotifyClientCredentials(client_id=client_ID, client_secret=client_secret)
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_ID, client_secret=client_secret))

    def search_song(title, artist, limit):
        try:
            results = sp.search(q=f"{title} {artist}", type='track', limit=limit)
            song_id = results['tracks']['items'][0]['id']
            return song_id
        except IndexError:
            print("Song not found!")
            return ""

    # Search for the song
    song_id = search_song(title, artist, 5)

    # Get audio features
    audio_features = sp.audio_features([song_id])
    song = pd.DataFrame(audio_features)

    # Data preprocessing
    features = song.select_dtypes(exclude=[object])
    columns_to_drop = ['liveness', 'duration_ms', 'time_signature']
    features.drop(columns=columns_to_drop, inplace=True)

    # Load scaler
    scaler_filename = "./feature_scaler.pickle"
    with open(scaler_filename, "rb") as file:
        scaler = pickle.load(file)

    # Transform audio features
    user_song_audio_features = scaler.transform(features)
    user_song_audio_features_df = pd.DataFrame(user_song_audio_features, columns=features.columns)

    # Load UMAP model
    umap_filename = "./umap.pickle"
    with open(umap_filename, "rb") as file:
        umap = pickle.load(file)

    # Transform with UMAP
    user_song_umap = umap.transform(user_song_audio_features_df)
    user_song_umap_df = pd.DataFrame(user_song_umap, columns=["UMAP_1", "UMAP_2"])

    # Load KMeans model
    cluster_model_filename = "./kmeans.pickle"
    with open(cluster_model_filename, "rb") as file:
        model = pickle.load(file)

    # Predict cluster
    user_song_cluster = model.predict(user_song_umap_df)[0]

    if song_id in data[data['hot_song'] == 1]['id'].tolist():
        result = data[(data['hot_song'] == 1) & (data['cluster'] == user_song_cluster)].sample(5)
    else:
        result = data[(data['hot_song'] != 1) & (data['cluster'] == user_song_cluster)].sample(5)

    result = pd.DataFrame(result)
    result['Spotify_Link'] = 'https://open.spotify.com/track/' + result['id']
    output = result[['Title', 'Artists', 'Spotify_Link']]

    print( )
    print('Here are songs you may like:')
    display(output)

# Call the function
recommend_songs()

Type in the artist:  prodigy
Type in the song name:  firestarter



Here are songs you may like:


,Title,Artists,Spotify_Link
2045,Willing To Trust,Kid Cudi,https://open.spotify.com/track/1OFQT3lHGbnn3x9...
2513,Running the Streets,Rick Ross,https://open.spotify.com/track/3s9f1LQ6607eDj9...
793,Droppin Jewels,Young Thug,https://open.spotify.com/track/3q8Uyfo98yoEqBy...
1615,We Can't Stop,Miley Cyrus,https://open.spotify.com/track/2y4lAQpi5VTNLu2...
1360,Fourth Of July,Fall Out Boy,https://open.spotify.com/track/1vrjcPplEuenckU...
